### Preparar ambiente

Primero descomentar las siguientes lineas e instalar PyAthena. Solo se hace la primera vez que se abre la instancia

In [1]:
import sys
!{sys.executable} -m pip install PyAthena

In [2]:
import sys
!{sys.executable} -m pip install pytz

#### Conectar con pyathena

In [3]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata'
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [4]:
fecha_inicio = '09-01-2020'
fecha_final = '09-30-2020'
fechas = pd.date_range(fecha_inicio, fecha_final)

# Union de tablas pings

In [5]:
historico_pings = f'historico_pings'
start_time = time.time()
for i,fecha in enumerate(fechas):
    time_fecha = time.time()
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    print(codigo_fecha)
    tabla_pings_fecha = f'todos_pings_delta_{codigo_fecha}'
    if i == 0 and (day == '01' and fecha.month == 2):
        query = f'''
                    CREATE TABLE {historico_pings}
                    WITH ( external_location = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-pings/{historico_pings}', 
                        format = 'PARQUET', 
                        parquet_compression = 'SNAPPY',
                        partitioned_by = ARRAY['iso_country_code']) AS
                    SELECT caid, 
                            utc_timestamp,
                            latitude,
                            longitude,
                            id_type,
                            hour_of_day,
                            year,
                            month,
                            day,
                            iso_country_code
                    FROM {tabla_pings_fecha}
                '''
    else:
        query = f'''
                    INSERT INTO {historico_pings} SELECT caid, 
                                                                utc_timestamp,
                                                                latitude,
                                                                longitude,
                                                                id_type,
                                                                hour_of_day,
                                                                year,
                                                                month,
                                                                day,
                                                                iso_country_code 
                                                        FROM {tabla_pings_fecha} 
                '''
    cursor.execute(query)
    tiempo = (time.time()-time_fecha)/60
    print(f"{codigo_fecha} insertada, {int(tiempo)} minutos")
print((time.time()-start_time)/60)

901
901 insertada, 1 minutos
902
902 insertada, 1 minutos
903
903 insertada, 1 minutos
904
904 insertada, 1 minutos
905
905 insertada, 1 minutos
906
906 insertada, 1 minutos
907
907 insertada, 1 minutos
908
908 insertada, 1 minutos
909
909 insertada, 1 minutos
910
910 insertada, 1 minutos
911
911 insertada, 1 minutos
912
912 insertada, 0 minutos
913
913 insertada, 0 minutos
914
914 insertada, 0 minutos
915
915 insertada, 0 minutos
916
916 insertada, 0 minutos
917
917 insertada, 0 minutos
918
918 insertada, 0 minutos
919
919 insertada, 0 minutos
920
920 insertada, 0 minutos
921
921 insertada, 0 minutos
922
922 insertada, 1 minutos
923
923 insertada, 1 minutos
924
924 insertada, 1 minutos
925
925 insertada, 1 minutos
926
926 insertada, 1 minutos
927
927 insertada, 1 minutos
928
928 insertada, 1 minutos
929
929 insertada, 1 minutos
930
930 insertada, 1 minutos
32.88312221368154


In [6]:
query = f'''MSCK REPAIR TABLE {historico_pings}'''
cursor.execute(query)